In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('brand_data/brand_count.csv')

In [18]:
df[df['name'] >= 30]

,brand,name
1,1028,386
15,3M,76
19,4U2,50
30,ABSOLUTE NEW YORK 絕色紐約,48
41,AHC,151
...,...,...
4047,黑人,156
4048,黑人牙膏,328
4050,黑松,99
4051,黑松 HeySong,223


In [19]:
df[df['name'] >= 30].sort_values('name', ascending=False).to_csv('brand_data_count.csv')

In [9]:
data = pd.read_csv('brand_data/brand_final.csv')

g:\Code\Python\GitHub\maboo\.venv\lib\site-packages\IPython\core\interactiveshell.py:3524: DtypeWarning: Columns (0,3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


ValueError: Can only compare identically-labeled Series objects